## Run Assertion Checks on Raw `exp01` Dataset

In [1]:
import numpy as np
from glob import glob
from datetime import datetime
from os.path import join, dirname, basename, getsize

from lib_analyze_exps import walk_one_level

In [2]:
### MODIFY BEGIN ###

# Specify the name of the experiment to be checked.
EXP_NAME = "exp01_nym-binaries-1.0.2_static-http-download"

# Specify path to raw experimental result files.
RAW_EXP_DIR_PARENT = "/PRIVATE_PATH/"

###  MODIFY END  ###

In [3]:
# Construct path to RAW_EXP_DIR: raw experimental result files repository.
RAW_EXP_DIR = join(RAW_EXP_DIR_PARENT, f"dataset_{EXP_NAME}_raw")

In [4]:
def check_script(file: str):
    
    print(f"-> Checking experiment script file '{basename(file)}'...")
    
    print("\n->-> Checking that we see the expected ID 'exp01' of the experiment to conduct in the script...")
    script_correct_exp_id = ! grep -c "^export mixcorr_exp_id=\"exp01\"$" {file}
    print(f"{script_correct_exp_id[0]=}")
    assert int(script_correct_exp_id[0]) == 1, f"{int(script_correct_exp_id[0])=} != 1"

In [5]:
def check_exp_log(results_dir: str, file: str):
    
    print(f"\n\n-> Checking experiment log file '{basename(file)}'...")
    
    print("\n->-> Checking that we see the right number of git tags checked out in the experiment scenarios repository...")
    exp_scens_correct_tag_num = ! grep -c "VERSION_FOR_{EXP_NAME}" {file}
    exp_scens_correct_tag_context = ! grep -Pcoz "HEAD detached at VERSION_FOR_{EXP_NAME}\nnothing to commit, working tree clean\nsending incremental file list\n" {file}
    print(f"{exp_scens_correct_tag_num[0]=}")
    print(f"{exp_scens_correct_tag_context[0]=}")
    assert int(exp_scens_correct_tag_num[0]) == 4, f"{int(exp_scens_correct_tag_num[0])=} != 4"
    assert int(exp_scens_correct_tag_context[0]) == 1, f"{int(exp_scens_correct_tag_context[0])=} != 1"
    
    print("\n->-> List all log lines regarding copying .patch files from their scenarios folder next to the Dockerfiles for image building...")
    exp_scens_patches_copied = ! grep -in "\.patch" {file}
    exp_scens_patches_copied = "\n".join(exp_scens_patches_copied)
    print(f"{exp_scens_patches_copied=}")
    assert "busybox/contracts_shorter-epoch.patch" in exp_scens_patches_copied, \
        "'busybox/contracts_shorter-epoch.patch' not in applied patches"
    assert "gateway/nym-gateway_track-traffic-flows.patch" in exp_scens_patches_copied, \
        "'gateway/nym-gateway_track-traffic-flows.patch' not in applied patches"
    assert "client_requester_server/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch" in exp_scens_patches_copied, \
        "'client_requester_server/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch' not in applied patches"
    assert "socks5-client/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch" in exp_scens_patches_copied, \
        "'socks5-client/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch' not in applied patches"
    
    assert "client_requester_server/nym-client_nym-socks5-client_disable-cover-traffic.patch" not in exp_scens_patches_copied, \
        "'client_requester_server/nym-client_nym-socks5-client_disable-cover-traffic.patch' applied as patch, but shouldn't have been"
    assert "socks5-client/nym-client_nym-socks5-client_disable-cover-traffic.patch" not in exp_scens_patches_copied, \
        "'socks5-client/nym-client_nym-socks5-client_disable-cover-traffic.patch' applied as patch, but shouldn't have been"
    assert "client_requester_server/nym-client_nym-socks5-client_increase-message-logging.patch" not in exp_scens_patches_copied, \
        "'client_requester_server/nym-client_nym-socks5-client_increase-message-logging.patch' applied as patch, but shouldn't have been"
    assert "socks5-client/nym-client_nym-socks5-client_increase-message-logging.patch" not in exp_scens_patches_copied, \
        "'socks5-client/nym-client_nym-socks5-client_increase-message-logging.patch' applied as patch, but shouldn't have been"
    
    print("\n->-> Match with `find` for .patch files in results folder subfolders...")
    exp_scens_patches_found = ! find {results_dir} -name "*\.patch" | sort -d
    print(*exp_scens_patches_found, sep = "\n")
    assert len(exp_scens_patches_found) == 4
    assert exp_scens_patches_found[0] == join(results_dir, "busybox/contracts_shorter-epoch.patch"), \
        f"{exp_scens_patches_found[0]} != {join(results_dir, 'busybox/contracts_shorter-epoch.patch')}"
    assert exp_scens_patches_found[1] == join(results_dir, "client_requester_server/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch"), \
        f"{exp_scens_patches_found[1]} != {join(results_dir, 'client_requester_server/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch')}"
    assert exp_scens_patches_found[2] == join(results_dir, "gateway/nym-gateway_track-traffic-flows.patch"), \
        f"{exp_scens_patches_found[2]} != {join(results_dir, 'gateway/nym-gateway_track-traffic-flows.patch')}"
    assert exp_scens_patches_found[3] == join(results_dir, "socks5-client/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch"), \
        f"{exp_scens_patches_found[3]} != {join(results_dir, 'socks5-client/nym-network-requester_nym-socks5-client_nymtech-nym-pull-1702_fix-buffer-reordering.patch')}"
    
    print("\n->-> Verifying that the bootstrapped Nym network is in epoch 1...")
    epoch_is_one = ! grep -A 8 "Logging current epoch (queried at nym_validator_api)..." {file}
    epoch_is_one = "\n".join(epoch_is_one)
    assert "\"id\": 1," in epoch_is_one, "Epoch ID is not equal to 1"
    assert "\"secs\": 180," in epoch_is_one, "Epoch duration is not equal to 180 seconds (which means our patch failed to apply)"
    
    print("\n->-> Verifying that our test gateway is present in the network...")
    gateway_details_file = join(results_dir, "nym-gateway-node-details.txt")
    gateway_id = ! grep "Identity Key: .*$" {gateway_details_file} | tr -s " " | cut -d " " -f 3
    assert len(gateway_id) == 1, f"Expected 1 gateway key file with its identity key, but found: {len(gateway_id)=} "
    gateway_present = ! grep "{gateway_id[0]}" {file}
    print(f"{gateway_present[0]=}")
    assert len(gateway_present) == 1, f"Expected to see the gateway's identity key once in '{file}', but found: {len(gateway_present)=}"
    assert gateway_present[0] == f"      \"identity_key\": \"{gateway_id[0]}\",", f"Unexpected gateway identity key line: '{gateway_present[0]}'"
    
    print("\n->-> Verifying that each mixnode is present on its respective layer in the network...")
    
    mixnode_one_details_file = join(results_dir, "nym-mixnode-binary-0-node-details.txt")
    mixnode_one_id = ! grep "Identity Key: .*$" {mixnode_one_details_file} | tr -s " " | cut -d " " -f 3
    assert len(mixnode_one_id) == 1, f"Expected 1 mixnode one key file with its identity key, but found: {len(mixnode_one_id)=} "
    mixnode_one_present = ! grep -B 8 "{mixnode_one_id[0]}" {file} | grep -e "layer" -e "identity_key"
    print(*mixnode_one_present, sep = "\n")
    assert len(mixnode_one_present) == 2, f"Expected to see 2 matches for mixnode one's 'layer' and 'identity_key', but found: {len(mixnode_one_present)=}"
    assert mixnode_one_present[0] == "      \"layer\": 1,", f"Unexpected 'layer' line for mixnode one: '{mixnode_one_present[0]}'"
    assert mixnode_one_present[1] == f"        \"identity_key\": \"{mixnode_one_id[0]}\",", f"Unexpected 'identity_key' line for mixnode one: '{mixnode_one_present[1]}'"
    
    mixnode_two_details_file = join(results_dir, "nym-mixnode-binary-1-node-details.txt")
    mixnode_two_id = ! grep "Identity Key: .*$" {mixnode_two_details_file} | tr -s " " | cut -d " " -f 3
    assert len(mixnode_two_id) == 1, f"Expected 1 mixnode two key file with its identity key, but found: {len(mixnode_two_id)=} "
    mixnode_two_present = ! grep -B 8 "{mixnode_two_id[0]}" {file} | grep -e "layer" -e "identity_key"
    print(*mixnode_two_present, sep = "\n")
    assert len(mixnode_two_present) == 2, f"Expected to see 2 matches for mixnode two's 'layer' and 'identity_key', but found: {len(mixnode_two_present)=}"
    assert mixnode_two_present[0] == "      \"layer\": 2,", f"Unexpected 'layer' line for mixnode two: '{mixnode_two_present[0]}'"
    assert mixnode_two_present[1] == f"        \"identity_key\": \"{mixnode_two_id[0]}\",", f"Unexpected 'identity_key' line for mixnode two: '{mixnode_two_present[1]}'"
    
    mixnode_three_details_file = join(results_dir, "nym-mixnode-binary-2-node-details.txt")
    mixnode_three_id = ! grep "Identity Key: .*$" {mixnode_three_details_file} | tr -s " " | cut -d " " -f 3
    assert len(mixnode_three_id) == 1, f"Expected 1 mixnode three key file with its identity key, but found: {len(mixnode_three_id)=} "
    mixnode_three_present = ! grep -B 8 "{mixnode_three_id[0]}" {file} | grep -e "layer" -e "identity_key"
    print(*mixnode_three_present, sep = "\n")
    assert len(mixnode_three_present) == 2, f"Expected to see 2 matches for mixnode three's 'layer' and 'identity_key', but found: {len(mixnode_three_present)=}"
    assert mixnode_three_present[0] == "      \"layer\": 3,", f"Unexpected 'layer' line for mixnode three: '{mixnode_three_present[0]}'"
    assert mixnode_three_present[1] == f"        \"identity_key\": \"{mixnode_three_id[0]}\",", f"Unexpected 'identity_key' line for mixnode three: '{mixnode_three_present[1]}'"
    
    print("\n->-> Checking that random file of correct size has been created...")
    exp_file_generated = ! grep -A 1 "Generating 1048566 random characters for experiment document of size 1048576 Bytes (10 remaining characters will be run-specific ID)..." {file}
    exp_file_generated = "\n".join(exp_file_generated)
    print(f"{exp_file_generated=}")
    assert "-rw-r--r-- 1 root root 1048566" in exp_file_generated, f"Unexpected permissions and/or size or generated file: '{exp_file_generated}'"

In [6]:
def check_docker_build_log(file: str, expected_compile_steps: int):
    
    print(f"\n\n-> Checking docker build log file '{basename(file)}'...")
    
    print("\n->-> Checking that `docker build` contains in-order steps to checkout nym git tag, copy patch files, apply patch files, and compile target...")
    build_steps_tag_patches_apply_compile = ! grep -e " : ARG " -e " : RUN " -e " : COPY " {file} | grep -e " : ARG mixcorr_nym_gittag" -e "git checkout tags/" -e "COPY empty \*\.patch /root/nym/" -e "RUN find /root/nym -name \"\*\.patch\" -exec patch -i" -e "cargo build"
    print(*build_steps_tag_patches_apply_compile, sep = "\n")
    
    if expected_compile_steps == 1:
        assert len(build_steps_tag_patches_apply_compile) == 5, f"{len(build_steps_tag_patches_apply_compile)=} != 5"
    elif expected_compile_steps == 2:
        assert len(build_steps_tag_patches_apply_compile) == 6, f"{len(build_steps_tag_patches_apply_compile)=} != 6"
    
    assert "ARG mixcorr_nym_gittag" in build_steps_tag_patches_apply_compile[0]
    assert " git checkout tags/\"${mixcorr_nym_gittag}\"" in build_steps_tag_patches_apply_compile[1]
    assert "COPY empty *.patch /root/nym/" in build_steps_tag_patches_apply_compile[2]
    assert "RUN find /root/nym -name \"*.patch\" -exec patch -i {} -p1 \;" in build_steps_tag_patches_apply_compile[3]
    assert "cargo build --manifest-path /root/nym/" in build_steps_tag_patches_apply_compile[4]
    if expected_compile_steps == 2:
        assert "cargo build --manifest-path /root/nym/" in build_steps_tag_patches_apply_compile[5]

In [7]:
def check_gateway_log(file: str):
    
    print(f"\n\n-> Checking docker run log file '{basename(file)}'...")
    
    print("\n->-> Check that no log line contains the message that stored messages were pushed to an endpoint...")
    pushed_stored_msgs_not_present = ! grep -c "\[MIXCORR\] \[push_stored_messages_to_client\] WARNING At least one on-disk message stored for endpoint" {file}
    print(f"{pushed_stored_msgs_not_present[0]=}")
    assert int(pushed_stored_msgs_not_present[0]) == 0, f"{int(pushed_stored_msgs_not_present[0])=} != 0"

In [8]:
def check_sphinxflow_logs(flows_dir: str):
    
    print(f"\n\n-> Checking SphinxFlow logs in '{flows_dir}'...")
    
    print("\n->-> Check that timestamps are monotonically increasing per SphinxFlow log file...")
    
    flow_files = sorted(glob(join(flows_dir, "*.sphinxflow")))
    
    for flow_file in flow_files:
        
        if getsize(flow_file) == 0:
            continue
        
        ts = np.loadtxt(fname = flow_file, dtype = np.uint64, delimiter = ",", skiprows = 1, usecols = 0)
        ts_monotonically_increasing = np.all(np.diff(ts) > 0)
        
        assert ts_monotonically_increasing == True, f"{ts_monotonically_increasing=} != True"

In [9]:
def check_runs(results_dir: str):
    
    print("\n\n-> Checking result folders of all runs in this experiment...")
    
    # Find all succeeded and failed runs.
    runs_succeeded_files = sorted(glob(join(results_dir, "exp01_curl_run_*", "SUCCEEDED")))
    runs_failed_files = sorted(glob(join(results_dir, "exp01_curl_run_*", "FAILED")))
    runs_completed_files = [ *runs_succeeded_files, *runs_failed_files ]
    
    assert len(runs_completed_files) == (len(runs_succeeded_files) + len(runs_failed_files)), \
        f"{len(runs_completed_files)=} != ({len(runs_succeeded_files)=} + {len(runs_failed_files)=})"
    
    # Convert lists with run indicators (SUCCEEDED or FAILED files) into lists containing
    # the files respective parent directories, i.e., the individual run folders.
    runs_succeeded = sorted(list(map(lambda file: dirname(file), runs_succeeded_files)))
    runs_failed = sorted(list(map(lambda file: dirname(file), runs_failed_files)))
    runs_completed = sorted(list(map(lambda file: dirname(file), runs_completed_files)))
    
    # Glob for nym-client configuration file in each SUCCEEDED run folder.
    client_confs = list(map(lambda run_dir: 
                            glob(join(run_dir, "client_nym_folder", "clients", "client_*", "config", "config.toml"))[0],
                            runs_succeeded))
    
    # Glob for nym-socks5client configuration file in each SUCCEEDED run folder.
    socks5client_confs = list(map(lambda run_dir:
                                  glob(join(run_dir, "socks5client_nym_folder", "socks5-clients", "socks5-client_*", "config", "config.toml"))[0],
                                  runs_succeeded))
    
    
    print("\n->-> Check that only a single result file (SUCCEEDED, FAILED) exists in each run directory...")
    runs_completed_unique = set(runs_completed)
    print(f"{len(runs_completed)=}")
    print(f"{len(runs_completed_unique)=}")
    assert len(runs_completed) == len(runs_completed_unique), f"{len(runs_completed)=} != {len(runs_completed_unique)=}"
    
    
    print("\n->-> Check that the SHA512 hashes of the curl client and webserver documents are equal...")
    webserver_doc_hashes = np.zeros(len(runs_succeeded), dtype = (np.unicode_, 128))
    
    for idx, run_succeeded_file in enumerate(runs_succeeded_files):
        
        with open(run_succeeded_file, mode = "r", encoding = "utf-8") as run_succeeded_fp:
            
            # Extract both documents' SHA512 hash from file (first part of each line).
            curl_doc_hash = str(run_succeeded_fp.readline().split(" ")[0])
            webserver_doc_hash = str(run_succeeded_fp.readline().split(" ")[0])
            assert curl_doc_hash == webserver_doc_hash, f"{curl_doc_hash=} != {webserver_doc_hash=}"
            
            # Store hash of webserver's document for later check.
            webserver_doc_hashes[idx] = webserver_doc_hash
    
    
    print("\n->-> Check that all found webserver documents are unique...")
    webserver_doc_hashes_unique = np.unique(webserver_doc_hashes)
    print(f"{len(webserver_doc_hashes)=}")
    print(f"{len(webserver_doc_hashes_unique)=}")
    assert len(webserver_doc_hashes_unique) == len(webserver_doc_hashes), \
        f"{len(webserver_doc_hashes_unique)=} != {len(webserver_doc_hashes)=}"
    assert webserver_doc_hashes_unique.size == webserver_doc_hashes.size, \
        f"{webserver_doc_hashes_unique.size=} != {webserver_doc_hashes.size=}"
    assert webserver_doc_hashes_unique.shape == webserver_doc_hashes.shape, \
        f"{webserver_doc_hashes_unique.shape=} != {webserver_doc_hashes.shape=}"
    
    
    print("\n->-> Check that FAILED runs fail for the single known failure reason...")
    for idx, run_failed_file in enumerate(runs_failed_files):
        
        with open(run_failed_file, mode = "r", encoding = "utf-8") as run_failed_fp:
            
            err_msg = str(run_failed_fp.read().strip())
            assert err_msg == "One endpoint initialization process of this run failed due to the validator-api being unavailable", \
                f"{err_msg=} != 'One endpoint initialization process of this run failed due to the validator-api being unavailable'"
    
    
    print("\n->-> Checking that all SUCCEEDED client configurations related to timing/delay values are the same...")
    
    print(f"{len(client_confs)=}")
    print(f"{len(socks5client_confs)=}")
    assert len(client_confs) == len(socks5client_confs), f"{len(client_confs)=} != {len(socks5client_confs)=}"
    
    # Ensure that all SUCCEEDED run nym-client configuration files
    # contain the expected part regarding timing and delay values.
    for client_conf in client_confs:
        
        with open(client_conf, mode = "r", encoding = "utf-8") as conf:
            
            conf_data = conf.read()
            assert "average_packet_delay = '50ms'\naverage_ack_delay = '50ms'\nloop_cover_traffic_average_delay = '200ms'\nmessage_sending_average_delay = '20ms'\n" in conf_data
    
    # Ensure that all SUCCEEDED run nym-socks5client configuration files
    # contain the expected part regarding timing and delay values.
    for socks5client_conf in socks5client_confs:
        
        with open(socks5client_conf, mode = "r", encoding = "utf-8") as conf:
            
            conf_data = conf.read()
            assert "average_packet_delay = '50ms'\naverage_ack_delay = '50ms'\nloop_cover_traffic_average_delay = '200ms'\nmessage_sending_average_delay = '20ms'\n" in conf_data

In [10]:
def check_failed_exps(results_dir: str, log_exp: str):
    
    print("\n\n-> Checking characteristics of FAILED runs...")
    
    # Find all runs of this experiment.
    results_dir_runs = join(results_dir, "exp01_curl_run_*")
    
    print("\n->-> Checking that the number of FAILED files, of \"validator-api unavailable\" error messages, and of experiment error log lines are equal...")
    runs_failed_num = ! find {results_dir} -path "*/exp01_curl_run_*/FAILED" | sort -d
    runs_failed_val_api_unavail = ! grep -rin "One endpoint initialization process of this run failed due to the validator-api being unavailable" {results_dir_runs}
    runs_failed_exp_log_matches = ! grep -rin "WARNING: The validator-api crashed! Restarting everything and trying again to reach" {log_exp}
    print(f"{runs_failed_num=}")
    print(f"{runs_failed_val_api_unavail=}")
    print(f"{runs_failed_exp_log_matches=}")
    assert len(runs_failed_num) == len(runs_failed_val_api_unavail) == len(runs_failed_exp_log_matches), \
        f"{len(runs_failed_num)=} ?!= {len(runs_failed_val_api_unavail)=} ?!= {len(runs_failed_exp_log_matches)=}"
    
    for idx, match in enumerate(runs_failed_num):
        
        # Extract the number of the FAILED run.
        run_num = dirname(match).rsplit(sep = "_", maxsplit = 1)[1]
        
        assert runs_failed_val_api_unavail[idx].startswith(match) == True, \
            f"Expected runs_failed_val_api_unavail[idx] to start with '{match}', but got: {runs_failed_val_api_unavail[idx]}"
        assert f" [run#{run_num}] " in runs_failed_exp_log_matches[idx], \
            f"' [run#{run_num}] ' not in {runs_failed_exp_log_matches[idx]=}"
    
    
    print("\n->-> Checking that we don't see the \"190 bytes left\" curl error messages anymore...")
    runs_failed_grep_curl = ! grep -rin "transfer closed with 190 bytes remaining to read" {results_dir} | wc -l
    print(f"{runs_failed_grep_curl[0]=}")
    assert int(runs_failed_grep_curl[0]) == 0, f"{int(runs_failed_grep_curl[0])=} != 0"

In [11]:
def check_exp_folder(path: str):
    
    script_exp = join(path, "2-bootstrap-nym-and-run-experiments.sh")
    log_exp = join(path, "logs_2-bootstrap-nym-and-run-experiments.log")
    
    log_build_bbox = join(path, "logs_docker-build_busybox.log")
    log_build_cli_req_srv = join(path, "logs_docker-build_client-requester-server.log")
    log_build_gw = join(path, "logs_docker-build_gateway.log")
    log_build_mix = join(path, "logs_docker-build_mixnode.log")
    log_build_socks5 = join(path, "logs_docker-build_socks5-client.log")
    log_build_val_api = join(path, "logs_docker-build_validator-api.log")
    
    log_run_gw_mix = join(path, "logs_docker-run_gateway-mixnodes.log")
    
    sphinxflow_logs = join(path, "gateway_sphinxflows")
    
    
    # Applying all check functions to the various log files.
    
    check_script(script_exp)
    
    check_exp_log(path, log_exp)
    check_docker_build_log(log_build_bbox, 4)
    check_docker_build_log(log_build_cli_req_srv, 5)
    check_docker_build_log(log_build_gw, 4)
    check_docker_build_log(log_build_mix, 4)
    check_docker_build_log(log_build_socks5, 4)
    check_docker_build_log(log_build_val_api, 4)
    
    check_gateway_log(log_run_gw_mix)
    
    check_sphinxflow_logs(sphinxflow_logs)
    
    check_runs(path)
    check_failed_exps(path, log_exp)

In [12]:
raw_exp_folders = walk_one_level(RAW_EXP_DIR, True)

In [13]:
print(f"----- [{datetime.now().strftime('%Y-%m-%d_%H:%M:%S.%f')}] Begin running assertion checks on raw exp01 dataset -----\n")

for raw_exp_folder in raw_exp_folders:
    
    print(f"\n\n----- BEGIN {raw_exp_folder} -----\n")
    check_exp_folder(raw_exp_folder)
    print(f"\n\n-----  END  {raw_exp_folder} -----\n\n\n")

print(f"----- [{datetime.now().strftime('%Y-%m-%d_%H:%M:%S.%f')}] Done running assertion checks on raw exp01 dataset -----\n")

----- [2022-12-22_10:18:53.986525] Begin running assertion checks on raw exp01 dataset -----



----- BEGIN /PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-16-56_mixcorr-nym-ccx32-exp02-one_exp01_nym-binaries-1.0.2_static-http-download -----

-> Checking experiment script file '2-bootstrap-nym-and-run-experiments.sh'...

->-> Checking that we see the expected ID 'exp01' of the experiment to conduct in the script...
script_correct_exp_id[0]='1'


-> Checking experiment log file 'logs_2-bootstrap-nym-and-run-experiments.log'...

->-> Checking that we see the right number of git tags checked out in the experiment scenarios repository...
exp_scens_correct_tag_num[0]='4'
exp_scens_correct_tag_context[0]='1'

->-> List all log lines regarding copying .patch files from their scenarios folder next to the Dockerfiles for image building...
exp_scens_patches_copied='146:busybox/contracts_shorter-epoch.patch\n148:client_requester_server/nym-network-requester_n

/tmp/ipykernel_2725/3310831638.py:14: UserWarning: loadtxt: input contained no data: "/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-17-24_mixcorr-nym-ccx32-exp02-two_exp01_nym-binaries-1.0.2_static-http-download/gateway_sphinxflows/CYUT72zGuGmvX3rrfeZvDbWFpUxkn49cpRUTPKRFYcCK_endpoint-to-gateway.sphinxflow"
  ts = np.loadtxt(fname = flow_file, dtype = np.uint64, delimiter = ",", skiprows = 1, usecols = 0)
/tmp/ipykernel_2725/3310831638.py:14: UserWarning: loadtxt: input contained no data: "/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-17-24_mixcorr-nym-ccx32-exp02-two_exp01_nym-binaries-1.0.2_static-http-download/gateway_sphinxflows/CYUT72zGuGmvX3rrfeZvDbWFpUxkn49cpRUTPKRFYcCK_gateway-to-endpoint.sphinxflow"
  ts = np.loadtxt(fname = flow_file, dtype = np.uint64, delimiter = ",", skiprows = 1, usecols = 0)



-> Checking result folders of all runs in this experiment...

->-> Check that only a single result file (SUCCEEDED, FAILED) exists in each run directory...
len(runs_completed)=3525
len(runs_completed_unique)=3525

->-> Check that the SHA512 hashes of the curl client and webserver documents are equal...

->-> Check that all found webserver documents are unique...
len(webserver_doc_hashes)=3523
len(webserver_doc_hashes_unique)=3523

->-> Check that FAILED runs fail for the single known failure reason...

->-> Checking that all SUCCEEDED client configurations related to timing/delay values are the same...
len(client_confs)=3523
len(socks5client_confs)=3523


-> Checking characteristics of FAILED runs...

->-> Checking that the number of FAILED files, of "validator-api unavailable" error messages, and of experiment error log lines are equal...
runs_failed_num=['/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-17-24_mixcorr-nym-ccx32-exp02-two_exp01_nym

/tmp/ipykernel_2725/3310831638.py:14: UserWarning: loadtxt: input contained no data: "/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-30-22_mixcorr-nym-ccx32-exp02-nine_exp01_nym-binaries-1.0.2_static-http-download/gateway_sphinxflows/F219MzMtRLjqvXqqtP4Czne8FvamxU1gyoyJC86ubqmx_endpoint-to-gateway.sphinxflow"
  ts = np.loadtxt(fname = flow_file, dtype = np.uint64, delimiter = ",", skiprows = 1, usecols = 0)
/tmp/ipykernel_2725/3310831638.py:14: UserWarning: loadtxt: input contained no data: "/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-30-22_mixcorr-nym-ccx32-exp02-nine_exp01_nym-binaries-1.0.2_static-http-download/gateway_sphinxflows/F219MzMtRLjqvXqqtP4Czne8FvamxU1gyoyJC86ubqmx_gateway-to-endpoint.sphinxflow"
  ts = np.loadtxt(fname = flow_file, dtype = np.uint64, delimiter = ",", skiprows = 1, usecols = 0)



-> Checking result folders of all runs in this experiment...

->-> Check that only a single result file (SUCCEEDED, FAILED) exists in each run directory...
len(runs_completed)=3515
len(runs_completed_unique)=3515

->-> Check that the SHA512 hashes of the curl client and webserver documents are equal...

->-> Check that all found webserver documents are unique...
len(webserver_doc_hashes)=3513
len(webserver_doc_hashes_unique)=3513

->-> Check that FAILED runs fail for the single known failure reason...

->-> Checking that all SUCCEEDED client configurations related to timing/delay values are the same...
len(client_confs)=3513
len(socks5client_confs)=3513


-> Checking characteristics of FAILED runs...

->-> Checking that the number of FAILED files, of "validator-api unavailable" error messages, and of experiment error log lines are equal...
runs_failed_num=['/PRIVATE_PATH/dataset_exp01_nym-binaries-1.0.2_static-http-download_raw/2022-12-05_14-30-22_mixcorr-nym-ccx32-exp02-nine_exp01_ny